## Задание 1:

Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156. Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик  20000 человек в день. Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .


In [133]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [134]:
# таймспент - количественная метрика

In [135]:
time_control, time_test = 25, 25 + 25 * .1
std_control, std_test = 156, 156 * 1.15
dau = 20_000

In [136]:
def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    es_import = effectsize_smd(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2)[0]
    
    return es_formula, es_import 

calc_continuous_effect_size(mean_1=time_control, std_1=std_control, mean_2=time_test, std_2=std_test) 

(-0.014871423685692022, -0.014870865944557932)

In [137]:
effect_size = calc_continuous_effect_size(mean_1=time_control, std_1=std_control, mean_2=time_test, std_2=std_test)[1]

def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)


selection = calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)
print(selection)

141971


In [138]:
duration = selection/dau
print(f'duration ~ {duration} days')

duration ~ 7.09855 days


## Задание 2:

Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта. Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.Средний трафик  4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%


In [139]:
# bounce rate в GA - качественная метрика

br_1, br_2 = .4, .32
dau = 4_000


In [140]:
def calc_proportion_es(prob1: float, prob2: float):
    return abs(proportion_effectsize(prob1, prob2)) 

es_prop = calc_proportion_es(br_1, br_2)
es_prop

0.16690997264630925

In [141]:
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)


selection = calc_sample_size_proportion(effect_size=es_prop, alpha=.05, beta=.2)
print(selection)

1126


In [142]:
duration = selection/dau
print(f'duration = {duration} day')

duration = 0.2815 day
